In [1]:
import ScraperFC as sfc
import traceback

In [2]:
scraper = sfc.Understat()
try:
    lg_table = scraper.scrape_league_table(year=2024, league='EPL')
except Exception as e:
    traceback.print_exc()
    raise
finally:
    scraper.close()

d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\ScraperFC\Understat.py:361: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  columns = pd.read_html(table)[0].columns
d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\ScraperFC\Understat.py:366: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table)[0]


In [3]:
understat_table = lg_table[['Team', 'W', 'D', 'L', 'G', 'GA', 'xG', 'xGA', 'NPxG',
                            'NPxGA', 'PPDA', 'OPPDA', 'PTS', 'xPTS']]
understat_table['season'] = 2324
understat_table['league'] = 'ENG-Premier League'

understat_table.loc[understat_table['Team'] == 'Manchester United', 'Team'] = 'Manchester Utd'
understat_table.loc[understat_table['Team'] == 'Newcastle United', 'Team'] = 'Newcastle Utd'
understat_table.loc[understat_table['Team'] == 'Tottenham Hotspur', 'Team'] = 'Tottenham'
understat_table.loc[understat_table['Team'] == 'Wolverhampton Wanderers', 'Team'] = 'Wolves'
understat_table.loc[understat_table['Team'] == 'Nottingham Forest', 'Team'] = "Nott'ham Forest"
understat_table.loc[understat_table['Team'] == 'Leicester', 'Team'] = 'Leicester City'
understat_table.loc[understat_table['Team'] == 'Leeds', 'Team'] = 'Leeds United'
understat_table.loc[understat_table['Team'] == 'Sheffield United', 'Team'] = "Sheffield Utd"
understat_table.loc[understat_table['Team'] == 'Luton', 'Team'] = 'Luton Town'
understat_table.loc[understat_table['Team'] == 'Norwich', 'Team'] = 'Norwich City'

understat_table.sort_values(by=['league', 'season', 'Team'], ascending=True, inplace=True)
understat_table.reset_index(drop=True, inplace=True)

understat_table.rename(columns={'Team': 'TEAM_NAME', 'W': 'TEAM_WINS', 'D': 'TEAM_DRAWS', 'league': 'COMPETITION',
                                'L': 'TEAM_LOSSES', 'xPTS': 'TEAM_XPTS', 'PTS': 'TEAM_PTS', 'season': 'SEASON',
                                'NPxGA': 'NPXG_AGAINST', 'xGA': 'XG_AGAINST'}, inplace=True)

understat_table.set_index(['COMPETITION', 'SEASON', 'TEAM_NAME'], inplace=True)

C:\Users\ksbha\AppData\Local\Temp\ipykernel_10208\395020011.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  understat_table['season'] = 2324
C:\Users\ksbha\AppData\Local\Temp\ipykernel_10208\395020011.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  understat_table['league'] = 'ENG-Premier League'
C:\Users\ksbha\AppData\Local\Temp\ipykernel_10208\395020011.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://p

In [4]:
import soccerdata as sd
import datetime
import pandas as pd

[12/29/23 16:51:50] INFO     No custom team name replacements found. You can configure these in       ]8;id=489203;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=504561;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\_config.py#84\84]8;;\
                             C:\Users\ksbha\soccerdata\config\teamname_replacements.json.                          

                    INFO     No custom league dict found. You can configure additional leagues in    ]8;id=695693;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=399914;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\_config.py#162\162]8;;\
                             C:\Users\ksbha\soccerdata\config\league_dict.json.                                    

In [5]:
def make_season_integer(df):
    indexes = df.index.names
    df = df.reset_index()
    df['SEASON'] = df['SEASON'].astype(int)
    df = df.set_index(indexes)
    return df

In [9]:
fbref = sd.FBref(leagues="ENG-Premier League", seasons=["2324"], no_cache=True)
fbref_team_standard = fbref.read_team_season_stats(stat_type='standard')

[12/29/23 16:57:51] INFO     Saving cached data to C:\Users\ksbha\soccerdata\data\FBref               ]8;id=196319;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=70028;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\_common.py#91\91]8;;\

In [62]:
fbref = sd.FBref(leagues="ENG-Premier League", seasons=["2324"])
fbref_team_standard = fbref.read_team_season_stats(stat_type='standard')

[12/25/23 18:16:39] INFO     Saving cached data to C:\Users\ksbha\soccerdata\data\FBref               ]8;id=306765;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=568265;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\_common.py#91\91]8;;\

In [98]:
fbref_team_standard = fbref_team_standard.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})

In [99]:
fbref_team_standard = make_season_integer(fbref_team_standard)

In [81]:
import pickle

In [45]:
pd.set_option('display.max_columns', None)

In [100]:
team_names = fbref_team_standard.reset_index()[['TEAM_NAME','url']]
team_names['TEAM_FBREF_ID'] = team_names['url'].apply(lambda x: x.split('/')[3])      ## Extracting team_id from URL
team_names.drop(columns=['url'], inplace=True)
team_names.columns = team_names.columns.droplevel(1)

C:\Users\ksbha\AppData\Local\Temp\ipykernel_9396\2026419047.py:3: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  team_names.drop(columns=['url'], inplace=True)


In [101]:
team_names['TEAM_FBREF_ID'] = team_names['TEAM_FBREF_ID'].astype('string')
team_names.drop_duplicates(inplace=True)

In [102]:
team_matches_played = fbref_team_standard['Playing Time'].MP

In [103]:
team_standard_stats = understat_table.copy()
team_standard_stats['MATCHES_PLAYED'] = team_matches_played

In [104]:
team_standard_stats.reset_index(inplace=True)

In [105]:
team_standard_stats = team_standard_stats.merge(team_names, on='TEAM_NAME', how='left')

In [106]:
team_standard_stats = team_standard_stats[['TEAM_FBREF_ID','SEASON','COMPETITION','MATCHES_PLAYED','TEAM_WINS', 
                                           'TEAM_DRAWS', 'TEAM_LOSSES', 'TEAM_PTS', 'TEAM_XPTS']]

In [82]:
pickle.dumps(fbref_team_standard)

b'\x80\x04\x95\xa7/\x00\x00\x00\x00\x00\x00\x8c\x11pandas.core.frame\x94\x8c\tDataFrame\x94\x93\x94)\x81\x94}\x94(\x8c\x04_mgr\x94\x8c\x1epandas.core.internals.managers\x94\x8c\x0cBlockManager\x94\x93\x94(\x8c\x16pandas._libs.internals\x94\x8c\x0f_unpickle_block\x94\x93\x94\x8c\x1apandas.core.arrays.integer\x94\x8c\x0cIntegerArray\x94\x93\x94)\x81\x94}\x94(\x8c\x05_data\x94\x8c\x15numpy.core.multiarray\x94\x8c\x0c_reconstruct\x94\x93\x94\x8c\x05numpy\x94\x8c\x07ndarray\x94\x93\x94K\x00\x85\x94C\x01b\x94\x87\x94R\x94(K\x01K\x14\x85\x94h\x15\x8c\x05dtype\x94\x93\x94\x8c\x02i8\x94\x89\x88\x87\x94R\x94(K\x03\x8c\x01<\x94NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00t\x94b\x89C\xa0\x17\x00\x00\x00\x00\x00\x00\x00\x19\x00\x00\x00\x00\x00\x00\x00\x19\x00\x00\x00\x00\x00\x00\x00\x1a\x00\x00\x00\x00\x00\x00\x00\x1b\x00\x00\x00\x00\x00\x00\x00\x1a\x00\x00\x00\x00\x00\x00\x00\x1a\x00\x00\x00\x00\x00\x00\x00\x18\x00\x00\x00\x00\x00\x00\x00\x19\x00\x00\x00\x00\x00\x00\x00\x18\x00\x00\x00\x00\x00\x00\x0

In [108]:
team_attacking_stats = understat_table.copy()

In [110]:
team_attacking_stats = team_attacking_stats.reset_index().merge(team_names, on='TEAM_NAME', how='left')

In [111]:
team_attacking_stats = team_attacking_stats[['TEAM_NAME','TEAM_FBREF_ID','COMPETITION','SEASON','NPxG','xG']]

In [112]:
team_attacking_stats.rename(columns={'NPxG': 'NPXG', 'xG':'XG'}, inplace=True)

In [114]:
team_attacking_stats.set_index(['COMPETITION', 'SEASON', 'TEAM_NAME'], inplace=True)

In [115]:
team_attacking_stats

TEAM_FBREF_ID   NPXG     XG
COMPETITION        SEASON TEAM_NAME                                  
ENG-Premier League 2324   Arsenal              18bb7c10  31.55  36.11
                          Aston Villa          8602292d  31.48  33.76
                          Bournemouth          4ba7cbea  26.93  26.93
                          Brentford            cd051869  28.61  30.89
                          Brighton             d07537b9  29.13  30.65
                          Burnley              943e8050  16.80  17.56
                          Chelsea              cff3d9bb  33.09  37.66
                          Crystal Palace       47c64c55  22.27  24.56
                          Everton              d3fd31cc  32.20   32.2
                          Fulham               fd962109  22.49  24.02
                          Liverpool            822bd0ba  34.75  37.95
                          Luton Town           e297cd13  18.39  19.92
                          Manchester City      b8fd03ef  35.22  38.26
                          Manchester Utd       19538871  25.15  27.45
                          Newcastle Utd        b2b47a98  37.63  39.92
                          Nott'ham Forest      e4a775cb  19.91  20.67
                          Sheffield Utd        1df6b87e  11.90  13.43
                          Tottenham            361ca564  32.70  33.46
                          West Ham             7c21e445  26.07  26.83
                          Wolves               8cec06e1  23.25  24.02

In [116]:
fbref_team_shooting = fbref.read_team_season_stats(stat_type='shooting')
fbref_team_passing = fbref.read_team_season_stats(stat_type='passing')
fbref_team_possession = fbref.read_team_season_stats(stat_type='possession')
fbref_team_misc = fbref.read_team_season_stats(stat_type='misc')

In [120]:
fbref_team_shooting = fbref_team_shooting.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_team_passing = fbref_team_passing.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_team_possession = fbref_team_possession.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_team_misc = fbref_team_misc.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})

In [127]:
fbref_team_shooting = make_season_integer(fbref_team_shooting)
fbref_team_passing = make_season_integer(fbref_team_passing)
fbref_team_possession = make_season_integer(fbref_team_possession)
fbref_team_misc = make_season_integer(fbref_team_misc)

In [128]:
team_goals_scored = understat_table.G
team_shots = fbref_team_shooting['Standard'].Sh
team_shotsOT = fbref_team_shooting['Standard'].SoT
team_pass_completed = fbref_team_passing["Total"].Cmp
team_pass_attempted = fbref_team_passing["Total"].Att
team_takeons_attempted = fbref_team_possession['Take-Ons'].Att
team_takeons_completed = fbref_team_possession['Take-Ons'].Succ
team_crossesintoPA = fbref_team_passing['CrsPA']
team_fouls_against = fbref_team_misc['Performance']['Fld']

In [129]:
team_shotsOT

COMPETITION         SEASON  TEAM_NAME      
ENG-Premier League  2324    Arsenal             87
                            Aston Villa         93
                            Bournemouth         82
                            Brentford           65
                            Brighton           108
                            Burnley             58
                            Chelsea             79
                            Crystal Palace      67
                            Everton             89
                            Fulham              74
                            Liverpool          101
                            Luton Town          44
                            Manchester City    107
                            Manchester Utd      73
                            Newcastle Utd       91
                            Nott'ham Forest     57
                            Sheffield Utd       42
                            Tottenham          108
                            West Ham  

In [130]:
team_attacking_stats['GOALS_SCORED'] = team_goals_scored
team_attacking_stats['SHOTS'] = team_shots
team_attacking_stats['SHOTS_ON_TARGET'] = team_shotsOT
team_attacking_stats['PASS_COMPLETED'] = team_pass_completed
team_attacking_stats['PASS_ATTEMPTED'] = team_pass_attempted
team_attacking_stats['TAKEONS_ATTEMPTED'] = team_takeons_attempted
team_attacking_stats['TAKEONS_COMPLETED'] = team_takeons_completed
team_attacking_stats['CROSSES_INTO_PA'] = team_crossesintoPA
team_attacking_stats['FOULS_AGAINST'] = team_fouls_against